In [ ]:
%pip install langchain
%pip install ctransformers
%pip install sentence-transformers
%pip install pinecone-client
%pip install pypdf
%pip install python_dotenv
%pip install langchain-pinecone
%pip install langchain-community
%pip install langchain-core
%pip install langchain-huggingface

In [ ]:
from langchain.chains import RetrievalQA 
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_pinecone import PineconeVectorStore as langchainpinecone
from langchain_community.document_loaders import PyPDFLoader,DirectoryLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.prompts import PromptTemplate
from langchain_community.llms import CTransformers
from pinecone import Pinecone
import os

In [ ]:
PINECONE_API_KEY="Enter your pinecone KEY here"

In [ ]:
def load_pdf(data):
    loader=DirectoryLoader(data,
                           glob="*.pdf",
                           loader_cls=PyPDFLoader)
    documents=loader.load()
    return documents

In [ ]:
extracted_data=load_pdf("data/")

In [ ]:
def text_spit(extracted_data):
    text_splitter=RecursiveCharacterTextSplitter(chunk_size=500,chunk_overlap=20)
    text_chunks=text_splitter.split_documents(extracted_data)
    return text_chunks